## Import Packages

In [5]:
import numpy as np
import pandas as pd
import glob
import os

## Import CSVs

In [6]:
# Current Path
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '/financial_data'
print(path)

csvfiles = glob.glob(path + "/*.csv")
# Creating a dataframe for each CSV file
dfs = [pd.read_csv(file) for file in csvfiles]


C:\Users\Dell\Documents\B.Sc IT (Hons.) Computing and Business\FYP - Luke Bezzina\Code\preprocessingHistoricalData/financial_data


# New Dataframe

In [7]:
processed_df_coll = {}

## Pre-Process

In [8]:
prev_row = None
# Up to 0.2% deviation from close price from day before is considered Neutral movement
neutral_percentage = 0.002

for df in dfs:
    processed_df = pd.DataFrame(columns = ['Date','d5_pd', 'd4_pd', 'd3_pd', 'd2_pd', 'd1_pd', 'Trend',
                                           'Volatility', 'PreviousClose', 'DayClose'])

    # Creating additional feature columns for each dataframe
    iterables = df.itertuples(index=True, name='Pandas')
    n = df.columns.get_loc('Name') # Name column in df
    df_name = df.iat[1, n] # Name of equity/etf

    # Per row in dataframe
    for i in range(11, len(df) ):
        close = df.columns.get_loc('Close')
        df_date = df.columns.get_loc('Date')

        # Price Trend Handling
        close_price = df.iat[i, close]
        prev_close_price = df.iat[i - 1, close]
        if close_price - prev_close_price > (close_price * neutral_percentage):
            direction = 'Positive'
        elif close_price - prev_close_price < - (close_price * neutral_percentage):
            direction = 'Negative'
        else:
            direction = 'Neutral'

        # Price Difference Handling
        d5_pd = df.iat[i - 5, close] - df.iat[0, close]
        d4_pd = df.iat[i - 4, close] - df.iat[0, close]
        d3_pd = df.iat[i - 3, close] - df.iat[0, close]
        d2_pd = df.iat[i - 2, close] - df.iat[0, close]
        d1_pd = df.iat[i - 1, close] - df.iat[0, close]
        date_ts = df.iat[i,df_date]

        pds = [d5_pd, d4_pd, d3_pd, d2_pd, d1_pd]

        volatility = np.std(pds)
        slope, intercept = np.polyfit([0, 1, 2, 3, 4], pds, 1)

        new_row = {'Date':date_ts, 'd5_pd':d5_pd, 'd4_pd':d4_pd, 'd3_pd':d3_pd, 'd2_pd':d2_pd,
                   'd1_pd':d1_pd, 'Trend':slope, 'Volatility':volatility, 'PreviousClose':prev_close_price, 'DayClose':close_price}
        #append row to the dataframe
        processed_df = processed_df.append(new_row, ignore_index=True)

    print(processed_df)
    processed_df_coll[df_name] = processed_df

ValueError: Expected 2D array, got scalar array instead:
array=0.6223375289985335.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

# Data Export

In [ ]:
for key, df in processed_df_coll.items():
    df.to_csv(key+".csv", index=False)

print("Export Complete!")